In [1]:
%pip install --upgrade python-dotenv nest_asyncio pydantic google-genai requests pandas

from IPython.display import clear_output ; clear_output()

In [6]:
import os
import random
import asyncio

from dotenv import load_dotenv
import nest_asyncio

from textwrap import dedent
from IPython.display import display, Markdown
def md(str): display(Markdown(str))

from pydantic import BaseModel, Field
from enum import Enum

import pandas as pd

from google import genai
from google.genai import types

load_dotenv()

nest_asyncio.apply()

gemini = genai.Client(api_key=os.getenv('GEMINI_API_KEY'))

G25PRO = 'gemini-2.5-pro-preview-03-25'
G25FLASH = 'gemini-2.5-flash-preview-04-17'

In [3]:
class Location(str, Enum):
    NATURE = 'nature'
    OFFICE = 'office'
    PUB = 'pub'
    STREET = 'street'
    HOME = 'home'

class Mood(str, Enum):
    HAPPY = 'happy'
    SAD = 'sad'
    CALM = 'calm'
    SCARY = 'scary'

class TimeOfDay(str, Enum):
    MORNING = 'morning'
    AFTERNOON = 'afternoon'
    EVENING = 'evening'
    NIGHT = 'night'

class Gender(str, Enum):
    MALE = 'male'
    FEMALE = 'female'
    OTHER = 'other'

class Person(BaseModel):
    name: str = Field(description="Name of the person")
    age: int = Field(description="Age of the person")
    gender: Gender = Field(description="Gender of the person")

class Story(BaseModel):
    people: list[Person] = Field(description="List of people in the story")
    location: Location = Field(description="Location of the story")
    time_of_day: TimeOfDay = Field(description="Time of day in the story")
    mood: Mood = Field(description="Mood of the story")

async def create_story():
    num_people = random.randint(2, 4)
    location = random.choice([loc.value for loc in Location])
    mood = random.choice([mood.value for mood in Mood])
    time_of_day = random.choice([time.value for time in TimeOfDay])

    prompt = dedent(f"""
        - Write a very short story depicting a situation.
        - Describe, the people (with their name, age, gender ({', '.join([gender.value for gender in Gender])}).
        - Describe the location, the time of day, and the mood with sufficient detail,
          to allow the reader to discern them clearly and unambiguously. The reader will be tested
          on their ability to identify the people, location, time of day, and mood from the story,
          so be sure to include enough detail for them to do so.
        - Don't use any special characters or markdown formatting.
        - The story should be roughly 200 tokens long.

        <story-specification>
          - Number of people: {num_people}
          - Location: {location}
          - Time of day: {time_of_day}
          - Mood: {mood}
        </story-specification>
    """)

    response = await gemini.aio.models.generate_content(
        model=G25FLASH,
        contents=prompt,
        config={
            'thinking_config': {'thinking_budget': 1000},
            'max_output_tokens': 1500,
            'temperature': 0.8,
        },
    )

    return prompt, response.text

async def parse_story(story):
    prompt = dedent(f"""
        Parse the story and extract the following information:
        - People: name, age, gender ({', '.join([gender.value for gender in Gender])})
        - Location: {', '.join([loc.value for loc in Location])}
        - Time of day: {', '.join([time.value for time in TimeOfDay])}
        - Mood: {', '.join([mood.value for mood in Mood])}

    <story>
      {story}
    </story>
    """)
    response = await gemini.aio.models.generate_content(
        model=G25FLASH,
        contents=prompt,
        config={
            'response_mime_type': 'application/json',
            'response_schema': Story,
            'thinking_config': {'thinking_budget': 3000},
            'temperature': 0.3,
        },
    )

    obj = response.parsed
    people = sorted(obj.people, key=lambda x: x.name)
    return dedent(f"""\
    PEOPLE:
    {'\n    '.join([f"  - {person.name}, {person.age}, {person.gender.value}" for person in people])}
    LOCATION: {obj.location.value}
    TIME_OF_DAY: {obj.time_of_day.value}
    MOOD: {obj.mood.value}
    """)

prompt, response = await create_story()
parsed = await parse_story(response)
md(f"**Prompt:**\n\n```{prompt}```\n\n**Response:**\n\n{response}\n\n**Parsed:**\n\n```\n{parsed}\n```")

**Prompt:**

```
- Write a very short story depicting a situation.
- Describe, the people (with their name, age, gender (male, female, other).
- Describe the location, the time of day, and the mood with sufficient detail,
  to allow the reader to discern them clearly and unambiguously. The reader will be tested
  on their ability to identify the people, location, time of day, and mood from the story,
  so be sure to include enough detail for them to do so.
- Don't use any special characters or markdown formatting.
- The story should be roughly 200 tokens long.

<story-specification>
  - Number of people: 4
  - Location: pub
  - Time of day: afternoon
  - Mood: scary
</story-specification>
```

**Response:**

The Crown and Anchor was unusually quiet. Dust motes danced in the single weak shaft of sunlight cutting through the gloom from a high window. John, a man in his early forties, nervously tapped his fingers on the sticky table. Across from him, Sarah, barely twenty, jumped at the creak of a floorboard upstairs. An older man, David, likely in his sixties, watched them with unsettlingly calm eyes from a corner booth, his face obscured by shadow. By the bar, Clara, a woman appearing to be John's contemporary, polished a glass with slow, deliberate strokes, her gaze fixed on something unseen. A sudden draft sent a shiver through the room. The air felt heavy, expectant, filled with a silent dread that made the hairs on the back of your neck stand up.

**Parsed:**

```
PEOPLE:
  - Clara, 42, female
  - David, 65, male
  - John, 42, male
  - Sarah, 20, female
LOCATION: pub
TIME_OF_DAY: evening
MOOD: scary

```

In [4]:
async def a_gen_item():
    _, response = await create_story()
    parsed = await parse_story(response)
    return response, parsed

dataset = await asyncio.gather(*[a_gen_item() for _ in range(100)])

In [ ]:
stories = pd.DataFrame(dataset, columns=['input', 'output'])
display(stories.head(3).style.set_properties(
    **{'text-align': 'left', 'white-space': 'pre-wrap', 'width': '50%'},
    # subset=['input', 'output'],
))

,input,output
0,"The Anchor pub was quiet, the late night hour having thinned the crowd considerably. Dim lights cast long, lonely shadows across the worn floorboards. At a corner table sat four people, a heavy sadness palpable around them. Alice, a woman of thirty five, stared down at her hands, her shoulders slumped. Ben, forty two and male, slowly sipped his drink, his gaze distant. Clara, a twenty eight year old female, quietly wiped a tear from her cheek. David, a man of fifty five, rested his forehead in his hand, a deep sigh escaping his lips. The silence between them was thick with sorrow, broken only by the occasional clinking glass elsewhere in the room.","PEOPLE: - Alice, 35, female - Ben, 42, male - Clara, 28, female - David, 55, male LOCATION: pub TIME_OF_DAY: night MOOD: sad"
1,"The quiet hung heavy in the living room. Eleanor, a woman of sixty-five, sat on the worn armchair, her eyes distant. Across from her, Robert, sixty-eight and male, stared at the unlit fireplace, his shoulders slumped. Sarah, thirty-five and female, gently stroked the hair of Tom, an eight-year-old boy, who was curled against her side on the sofa, his small face buried in her shirt. The grey light filtering through the undrawn curtains confirmed it was early morning. A single unpacked box sat in the corner, a silent reminder of why the air felt so thick with sorrow. No one spoke, the shared grief a palpable weight in the room.","PEOPLE: - Eleanor, 65, female - Robert, 68, male - Sarah, 35, female - Tom, 8, male LOCATION: home TIME_OF_DAY: morning MOOD: sad"
2,"Elias, a male aged 68, walked slowly beside Clara, a female aged 72. They ambled down Elm Street, the smooth grey pavement warm beneath their worn shoes. A few cars were parked quietly along the curb, and mature oak trees cast long, gentle shadows across the sidewalk. The sun, lower in the sky now, bathed everything in a soft, golden light typical of late afternoon. Their conversation was quiet, punctuated by comfortable silences. A gentle breeze rustled the leaves overhead. There was no hurry in their steps, only a shared sense of peace as they enjoyed the simple act of being together in the quiet afternoon light. The air felt still and warm around them.","PEOPLE: - Clara, 72, female - Elias, 68, male LOCATION: street TIME_OF_DAY: afternoon MOOD: calm"
